In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

import os

In [2]:
DATA_DIR = 'datasets'
datasets = {}
for f in os.listdir(DATA_DIR):
    if f.endswith('.csv'):
        datasets[f.replace('.csv', '')] = pd.read_csv(os.path.join(DATA_DIR, f))

In [3]:
datasets.keys()

['part_categories',
 'themes',
 'colors',
 'parts',
 'inventory_parts',
 'sets',
 'inventory_sets',
 'inventories']

### Schema Information

![alt text](downloads_schema.png)


Lets try to answer the following question
1. The average number of parts used in lego sets per year
2. How the color spectrum changed over the years
3. Stack bar chart showing composition of different types of part type by theme - top 100 theme by part usage
4. Word cloud for different part categories based on part name

Machine learning questions
1. Given the part description can we predict the part type
2. Set Theme prediction using colors and parts used

In [4]:
print datasets['sets'].year.min()
print datasets['sets'].year.max()

1950
2017


### Data Visualizations

In [104]:
from bokeh.models import (HoverTool, FactorRange, Plot, LinearAxis, Grid,
                          Range1d)
from bokeh.io import show, output_notebook
from bokeh.models.glyphs import VBar
from bokeh.plotting import figure
from bokeh.embed import components
from bokeh.models.sources import ColumnDataSource
from bokeh.core.properties import value
from bokeh.palettes import viridis, mpl, brewer, d3
output_notebook()

Loading BokehJS ...

In [125]:
def create_hover_tool():
    """Generates the HTML for the Bokeh's hover data tool on our graph."""
    hover_html = """
      <div>
        <span class="hover-tooltip">$x{1}</span>
      </div>
      <div>
        <span class="hover-tooltip">$y</span>
      </div>
    """
    return HoverTool(tooltips=hover_html)

def plot_bar_chart(df, x_name, y_name, y_label, x_label, width, height):
    df[x_name] = df[x_name].astype(str)
    plot = figure(x_range = df[x_name].values, 
                  plot_width=width, plot_height=height, h_symmetry=False,
                  v_symmetry=False, tools="pan,wheel_zoom,box_zoom,reset",
                  min_border=0, toolbar_location="above",
                  responsive=True, outline_line_color="#666666")
    source = ColumnDataSource(df)
    plot.vbar(x=x_name,top=y_name, width=0.5, source = source)
    xaxis, yaxis = LinearAxis(), LinearAxis()
    plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
    plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))
    plot.toolbar.logo = None
    plot.min_border_top = 0
    plot.xgrid.grid_line_color = None
    plot.ygrid.grid_line_color = "#999999"
    plot.yaxis.axis_label = y_label
    plot.ygrid.grid_line_alpha = 0.1
    plot.xaxis.axis_label = x_label
    plot.xaxis.major_label_orientation = 1
    plot.add_tools(HoverTool(tooltips=[(x_label, '@'+x_name), (y_label, '@'+y_name)]))
    return plot

def getTopNEntries(df, topN, x_name, y_name, final_agg = 'mean'):
    if df.shape[0] <= topN:
        return df
    lengthN = True
    df_temp = df.copy().reset_index(drop=True)
    if final_agg != 'mean':
        lengthN = False
        df_temp.loc[topN] = ['others', df_temp[topN:][y_name].agg('sum')]
    return df_temp.loc[:(topN - 1) if lengthN else topN].copy()

def plot_agg_bar(df_set, x_name, y_name, width=1200, height=300, 
                       year_range = (1950, 2017), orderBy = False,
                       orderType = True,
                       TopX = None, X_label_name = None, y_label_name = None,
                       agg = 'mean'):
    X_label_name = x_name if X_label_name is None else X_label_name
    y_label_name = y_name if y_label_name is None else y_label_name    
    data = pd.DataFrame(df_set.groupby(x_name).agg({y_name: agg})).reset_index()
    if orderBy: 
        data.sort_values(y_name, inplace=True, ascending = orderType)
    if TopX is not None:
        data = getTopNEntries(data, TopX, x_name, y_name, agg)
    return plot_bar_chart(data, x_name, y_name, y_label_name, X_label_name, width, height)
    

In [126]:
show(plot_agg_bar(datasets['sets'], 'year', 'num_parts',
                  X_label_name = "Year", 
                  y_label_name = 'Avg number of Parts', 
                  agg = 'mean'))

/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: responsive parameter was deprecated in Bokeh 0.12.10 and will be removed, use sizing_mode='fixed' for responsive=False or sizing_mode='scale_width' for responsive=True instead.
  warn(message)


In [141]:
#Color plot by year
#from numpy.random import random
import numpy as np
from bokeh.core.properties import value
import logging
logging.basicConfig()

def plot_bar_stacked_chart(df, x_name, y_label, x_label, width, height, useColsColr = False, createLegend = False):
    stackCol = list(df.columns.values)
    
    colorPallette = np.array(mpl['Plasma'][256])
    np.random.shuffle(colorPallette)
    colors = map(lambda x: '#' + x.lower(), df.columns.values) if useColsColr else d3['Category20'][20]#list(colorPallette)[:len(stackCol)]
    #
    legends = [value(x) for x in stackCol] if createLegend else None
    
    df = df.reset_index()
    tooltipVals = map(lambda x: (x.replace(',', ''), '@{'+ x +'}'), df.columns.values)
    df[x_name] = df[x_name].astype(str)
    source = ColumnDataSource(data=df)
    x_values = map(lambda x: str(x), df[x_name].values)
    plot = figure(x_range = x_values, plot_width=width, plot_height=height, 
                  tools="pan,wheel_zoom,box_zoom,reset",
                  min_border=0, outline_line_color="#666666")
    
    plot.vbar_stack(stackCol, x=x_name, width=0.5, color=colors, source=source,
                    legend = legends)
    xaxis, yaxis = LinearAxis(), LinearAxis()
    plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
    plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))
    plot.xgrid.grid_line_color = None
    plot.ygrid.grid_line_color = "#999999"
    plot.yaxis.axis_label = y_label
    plot.ygrid.grid_line_alpha = 0.1
    plot.xaxis.axis_label = x_label
    plot.xaxis.major_label_orientation = 1
    plot.add_tools(HoverTool(tooltips=tooltipVals))
    return plot



In [128]:
TopNColor = 50
yearColordf = pd.merge(pd.merge(pd.merge(datasets['sets'], datasets['inventories'], on='set_num').rename(columns = {'id': 'inventory_id'}), 
                                datasets['inventory_parts'], on = 'inventory_id'), datasets['colors'], 
                                left_on = 'color_id', right_on = 'id' )

topColors = yearColordf.groupby('rgb')['quantity'].sum().reset_index().sort_values(by='quantity', ascending=False).reset_index(drop=True)[:TopNColor]
groupedData = yearColordf[yearColordf.rgb.isin(topColors.rgb)][['year', 'rgb', 'quantity']].groupby(['year', 'rgb']).quantity.sum().reset_index()
pivot_data = groupedData.pivot(index = 'year', values = 'quantity', columns = 'rgb').fillna(0)

p = plot_bar_stacked_chart(pivot_data, 'year', 'Colors', 'Year', 1200, 300, useColsColr = True)
show(p)

In [142]:
#Parts requirement by Theme top 100 theme by count
TopNTheme = 50
TopNPartType = 20
setPartDf = pd.merge(pd.merge(pd.merge(datasets['sets'][['set_num', 'theme_id']], datasets['inventories'][['set_num', 'id']], on='set_num').rename(columns = {'id': 'inventory_id'}), 
                              datasets['inventory_parts'][['inventory_id', 'part_num', 'quantity']], on = 'inventory_id'), datasets['parts'][['part_cat_id', 'part_num']], 
                              on ='part_num')
setPartDf = pd.merge(setPartDf, datasets['part_categories'], left_on = 'part_cat_id', right_on = 'id').drop(['id'], 1).rename(columns = {'name': 'part_cat_name'})
ThemePartDf = pd.merge(setPartDf, datasets['themes'], left_on = 'theme_id', right_on = 'id')
topThemes = ThemePartDf.groupby('name')['id'].count().reset_index().rename(columns = {'id': 'count'}).sort_values(by='count', ascending=False)[:TopNTheme]
topPartType = ThemePartDf.groupby('part_cat_name')['id'].count().reset_index().rename(columns = {'id': 'count'}).sort_values(by='count', ascending=False)[:TopNPartType]
ThemePartTypeCount = ThemePartDf[(ThemePartDf.name.isin(topThemes.name)) & (ThemePartDf.part_cat_name.isin(topPartType.part_cat_name))]
ThemePartTypeCount = ThemePartTypeCount.groupby(['name', 'part_cat_name'])['quantity'].sum().reset_index()
ThemePartTypeCount_pivot = ThemePartTypeCount.pivot(index = 'name', values = 'quantity', columns = 'part_cat_name').fillna(0)
p = plot_bar_stacked_chart(ThemePartTypeCount_pivot, 'name', 'Colors', 'Year', 800, 500, createLegend=True)
p.legend.label_text_font_size = "5pt"
p.legend.glyph_height= 5
p.legend.glyph_width= 5
p.legend.label_height= 5
p.legend.label_width= 5
show(p)

[('name', '@{name}'), ('Bars Ladders and Fences', '@{Bars, Ladders and Fences}'), ('Bricks', '@{Bricks}'), ('Bricks Curved', '@{Bricks Curved}'), ('Bricks Round and Cones', '@{Bricks Round and Cones}'), ('Bricks Sloped', '@{Bricks Sloped}'), ('Bricks Special', '@{Bricks Special}'), ('Hinges Arms and Turntables', '@{Hinges, Arms and Turntables}'), ('Minifig Accessories', '@{Minifig Accessories}'), ('Minifigs', '@{Minifigs}'), ('Plants and Animals', '@{Plants and Animals}'), ('Plates', '@{Plates}'), ('Plates Round and Dishes', '@{Plates Round and Dishes}'), ('Plates Special', '@{Plates Special}'), ('Technic Axles', '@{Technic Axles}'), ('Technic Beams', '@{Technic Beams}'), ('Technic Connectors', '@{Technic Connectors}'), ('Technic Pins', '@{Technic Pins}'), ('Tiles', '@{Tiles}'), ('Tiles Special', '@{Tiles Special}'), ('Wheels and Tyres', '@{Wheels and Tyres}')]


In [130]:
ThemePartTypeCount_pivot.head()

part_cat_name,"Bars, Ladders and Fences",Bricks,Bricks Curved,Bricks Round and Cones,Bricks Sloped,Bricks Special,"Hinges, Arms and Turntables",Minifig Accessories,Minifigs,Plants and Animals,Plates,Plates Round and Dishes,Plates Special,Technic Axles,Technic Beams,Technic Connectors,Technic Pins,Tiles,Tiles Special,Wheels and Tyres
name,,,,,,,,,,,,,,,,,,,,
9V,254.0,1497.0,148.0,362.0,855.0,796.0,532.0,251.0,399.0,82.0,3072.0,329.0,630.0,36.0,32.0,45.0,93.0,867.0,344.0,403.0
Airport,319.0,2163.0,408.0,446.0,1842.0,414.0,972.0,608.0,731.0,186.0,6417.0,1299.0,1272.0,1057.0,770.0,931.0,1746.0,1448.0,577.0,1734.0
Basic,343.0,14834.0,156.0,173.0,1289.0,158.0,112.0,46.0,179.0,623.0,1196.0,93.0,121.0,20.0,0.0,0.0,12.0,29.0,7.0,798.0
Basic Set,532.0,71761.0,778.0,1017.0,8975.0,807.0,364.0,187.0,341.0,710.0,6137.0,985.0,686.0,25.0,2.0,4.0,19.0,256.0,174.0,1934.0
Batman,571.0,1814.0,907.0,752.0,2285.0,876.0,516.0,962.0,970.0,206.0,3965.0,1623.0,2428.0,374.0,275.0,403.0,1205.0,1718.0,765.0,466.0


In [122]:
setPartDf = pd.merge(pd.merge(pd.merge(pd.merge(datasets['sets'][['set_num', 'theme_id',
                                                            'year', 'num_parts']],
                                            datasets['inventories'][['set_num', 'id']],
                                            on='set_num').rename(columns = {'id': 'inventory_id'}),
                                  datasets['inventory_parts'][['inventory_id',
                                                                'part_num',
                                                                'quantity',
                                                                'color_id']],
                                  on = 'inventory_id'), datasets['colors'], 
                         left_on = 'color_id', right_on = 'id').drop(['id', 'name'], 1)
                         , datasets['parts'][['part_cat_id', 'part_num']],
                        on ='part_num')
setPartDf = pd.merge(setPartDf, datasets['part_categories'], left_on = 'part_cat_id', right_on = 'id').drop(['id'], 1).rename(columns = {'name': 'part_cat_name'})
ThemePartDf = pd.merge(setPartDf, datasets['themes'], left_on = 'theme_id', right_on = 'id')
ThemePartDf.head()

,set_num,theme_id,year,num_parts,inventory_id,part_num,quantity,color_id,rgb,is_trans,part_cat_id,part_cat_name,id,name,parent_id
0,00-1,414,1970,471,5574,29c01,8,4,C91A09,f,16,Windows and Doors,414,Castle,411.0
1,00-1,414,1970,471,5574,29c01,6,15,FFFFFF,f,16,Windows and Doors,414,Castle,411.0
2,00-1,414,1970,471,5574,3081cc01,2,15,FFFFFF,f,16,Windows and Doors,414,Castle,411.0
3,00-1,414,1970,471,5574,3001a,9,4,C91A09,f,11,Bricks,414,Castle,411.0
4,00-1,414,1970,471,5574,3001a,25,15,FFFFFF,f,11,Bricks,414,Castle,411.0


In [9]:
#Average number of parts for the top x themes
plot_df = pd.merge(datasets['sets'], datasets['themes'], left_on = 'theme_id', right_on = 'id')
show(plot_agg_bar(plot_df, 'name_y', 'num_parts',
                  X_label_name = "Theme Name", 
                  y_label_name = 'Average number of parts', 
                  orderBy = True, orderType = False, TopX = 50,
                  width=600, height=300,
                  agg = 'mean'))

/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: responsive parameter was deprecated in Bokeh 0.12.10 and will be removed, use sizing_mode='fixed' for responsive=False or sizing_mode='scale_width' for responsive=True instead.
  warn(message)


In [10]:
#Count of parts by category
plot_df = pd.merge(datasets['parts'], datasets['part_categories'], left_on = 'part_cat_id', right_on = 'id')
show(plot_agg_bar(plot_df, 'name_y', 'part_num',
                  X_label_name = "Theme Name", 
                  y_label_name = 'Average number of parts',
                  agg = 'count', width=600, height=300, ))

/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: responsive parameter was deprecated in Bokeh 0.12.10 and will be removed, use sizing_mode='fixed' for responsive=False or sizing_mode='scale_width' for responsive=True instead.
  warn(message)


TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('S32') dtype('S32') dtype('S32')

In [44]:
#groupedData.pivot(index = 'year', values = 'quantity', columns = 'rgb').fillna(0).columns.values
groupedData

,year,rgb,quantity


### Data Modeling

#### Part type prediction by part description

In [9]:
ml_df = pd.merge(datasets['parts'], datasets['part_categories'], left_on = 'part_cat_id', right_on = 'id')
ml_df.rename(columns = {'name_x': 'part_desc', 'name_y': 'part_name'}, inplace = True)
ml_df.head()

,part_num,part_desc,part_cat_id,id,part_name
0,0687b1,Set 0687 Activity Booklet 1,17,17,Non-LEGO
1,10016414,Sticker Sheet #1 for 41055-1,17,17,Non-LEGO
2,10019stk01,Sticker for Set 10019 - (43274/4170393),17,17,Non-LEGO
3,10026stk01,Sticker for Set 10026 - (44942/4184185),17,17,Non-LEGO
4,10029stk01,Sticker for Set 10029 - (4216816),17,17,Non-LEGO


In [47]:
# As all the words are potentially descriptive we don't any sort of preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, log_loss, roc_auc_score
from datetime import datetime
from scipy import sparse
from sklearn.linear_model import LogisticRegression

In [20]:
class nbsvm():
    def pr(self, y_i, y, data):
        p = data[y==y_i].sum(0)
        return (p+1) / ((y==y_i).sum()+1)

    def fit(self, data, y):
        y = y.values
        #print 'y shape', y.shape
        r = sparse.csr_matrix(np.log(self.pr(1,y, data) / self.pr(0,y, data)))
        m = LogisticRegression(C=4, dual=True)
        x_nb = data.multiply(r)
        #np.multiply(r.reshape(1, len(r)), data.values)
        #print np.isnan(x_nb).any(), np.isnan(y).any(), np.isinf(x_nb).any(), np.isinf(y).any()
        self.model = (m.fit(x_nb, y), r)
        return self.model

    def predict_proba(self, data):
        m, r = self.model
        #return m.predict_proba(np.multiply(r.reshape(1, len(r)), data.values))
        return m.predict_proba(data.multiply(r))
    
def evaluate(y_true, y_pred):
    return {
       'accuracy Score': accuracy_score(y_true, y_pred)
    }

In [37]:
vec = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
                      smooth_idf=1, sublinear_tf=1)
#using 10% of the data for test purpose
train_x, test_x, train_y, test_y = train_test_split(ml_df['part_desc'], ml_df['part_name'], test_size = 0.1)
trn_term_doc = vec.fit_transform(train_x)
test_term_doc = vec.transform(test_x)


models = [('RFC', RandomForestClassifier(n_estimators = 500, max_depth = 30))]
for mdlName, mdl in models:
    print('Model Name', mdlName)
    mdl.fit(trn_term_doc, train_y.reset_index(drop=True))
    pred = mdl.predict(test_term_doc)
    print evaluate(test_y, pred)

('Model Name', 'RFC')
{'accuracy Score': 0.7442307692307693}


In [48]:
#one vs rest modeling
labels = {}
for name in ml_df['part_name'].unique():
    labels[name] = (ml_df['part_name'] == name).astype(int)
labelEnc = pd.DataFrame(labels)
train_x, test_x, train_y, test_y = train_test_split(ml_df['part_desc'], labelEnc, test_size = 0.1)
trn_term_doc = vec.fit_transform(train_x)
test_term_doc = vec.transform(test_x)


models = [('nbsvm', nbsvm())]#, ('extraTreeClassifier', ExtraTreesClassifier(n_jobs=-1, random_state=3))]
for mdlName, mdl in models:
    preds = np.zeros((test_term_doc.shape[0], labelEnc.shape[1]))
    print('Model Name', mdlName)
    for i, j in enumerate(labelEnc.columns):
        print('fit', j)
        mdl.fit(trn_term_doc, train_y[j].reset_index(drop=True))
        preds[:,i] = mdl.predict_proba(test_term_doc)[:,1]

('Model Name', 'nbsvm')
('fit', 'Bars, Ladders and Fences')
('fit', 'Baseplates')
('fit', 'Belville, Scala and Fabuland')
('fit', 'Bionicle, Hero Factory and Constraction')
('fit', 'Bricks')
('fit', 'Bricks Curved')
('fit', 'Bricks Printed')
('fit', 'Bricks Round and Cones')
('fit', 'Bricks Sloped')
('fit', 'Bricks Special')
('fit', 'Bricks Wedged')
('fit', 'Clikits')
('fit', 'Containers')
('fit', 'Duplo, Quatro and Primo')
('fit', 'Flags, Signs, Plastics and Cloth')
('fit', 'HO Scale')
('fit', 'Hinges, Arms and Turntables')
('fit', 'Magnets and Holders')
('fit', 'Mechanical')
('fit', 'Minifig Accessories')
('fit', 'Minifigs')
('fit', 'Non-Buildable Figures (Duplo, Fabuland, etc)')
('fit', 'Non-LEGO')
('fit', 'Other')
('fit', 'Panels')
('fit', 'Plants and Animals')
('fit', 'Plates')
('fit', 'Plates Angled')
('fit', 'Plates Round and Dishes')
('fit', 'Plates Special')
('fit', 'Pneumatics')
('fit', 'Power Functions, Mindstorms and Electric')
('fit', 'Rock')
('fit', 'String, Bands and Ree

In [53]:
def oneVRestEval(true_y, pred_y):
    lossValue = []
    for i in range(true_y.shape[0]):
        lossValue.append(roc_auc_score(true_y[:, i], pred_y[:, i]))
    return np.array(lossValue).mean()

oneVRestEval(test_y.values, preds)

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [77]:
df1 = pd.DataFrame(data = {'name': range(10), 'ran': range(10)})
#pd.append(df1.loc[:5], pd.DataFrame(data = {'name': 'other', 'ran': df1[6:]['ran'].sum()}))

In [83]:
df1.loc[:4]['ran'].agg('mean')

2.0

,set_num,theme_id,year,num_parts,inventory_id,part_num,quantity,part_cat_id,part_cat_name,id,name,parent_id
0,00-1,414,1970,471,5574,29c01,8,16,Windows and Doors,414,Castle,411.0
1,00-1,414,1970,471,5574,29c01,6,16,Windows and Doors,414,Castle,411.0
2,00-1,414,1970,471,5574,3081cc01,2,16,Windows and Doors,414,Castle,411.0
3,00-1,414,1970,471,5574,3001a,25,11,Bricks,414,Castle,411.0
4,00-1,414,1970,471,5574,3001a,9,11,Bricks,414,Castle,411.0
5,00-1,414,1970,471,5574,3001a,4,11,Bricks,414,Castle,411.0
6,00-1,414,1970,471,5574,3002a,6,11,Bricks,414,Castle,411.0
7,00-1,414,1970,471,5574,3002a,16,11,Bricks,414,Castle,411.0
8,00-1,414,1970,471,5574,3003,5,11,Bricks,414,Castle,411.0
9,00-1,414,1970,471,5574,3003,3,11,Bricks,414,Castle,411.0


In [77]:
datasets['themes']

,id,name,parent_id
0,1,Technic,NaN
1,2,Arctic Technic,1.0
2,3,Competition,1.0
3,4,Expert Builder,1.0
4,5,Model,1.0
5,6,Airport,5.0
6,7,Construction,5.0
7,8,Farm,5.0
8,9,Fire,5.0
9,10,Harbor,5.0
